In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# SET UP

In [2]:
!git clone -l -s https://github.com/yu45020/Waifu2x.git waifu2x
%cd waifu2x

Cloning into 'waifu2x'...
remote: Enumerating objects: 546, done.
remote: Total 546 (delta 0), reused 0 (delta 0), pack-reused 546
Receiving objects: 100% (546/546), 138.07 MiB | 30.84 MiB/s, done.
Resolving deltas: 100% (284/284), done.
/content/waifu2x


In [3]:
%cd model_check_points/Upconv_7/
!7z e anime.7z
%cd ../CRAN_V2/
!unzip -qq -o CRAN_V2_02_28_2019.zip
%cd ../vgg_7/
!7z e art.7z
%cd ../../

/content/waifu2x/model_check_points/Upconv_7

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 19867323 bytes (19 MiB)

Extracting archive: anime.7z
--
Path = anime.7z
Type = 7z
Physical Size = 19867323
Headers Size = 279
Method = BZip2
Solid = +
Blocks = 1

  0%      6% 1 - anime/noise0_scale2.0x_model.json                                           13% 1 - anime/noise0_scale2.0x_model.json                                           18% 2 - anime/noise1_model.json                                 27% 3 - anime/noise1_scale2.0x_mode

In [4]:
import os

import cv2
from urllib.request import urlretrieve
from google.colab.patches import cv2_imshow

from torchvision.utils import save_image
from utils.prepare_images import *
from Models import *

# Getting Data

In [5]:
crop_path = '/content/drive/MyDrive/23-1 CV프로젝트 1팀/add_webtoon/crop/' # 크롭된 이미지있는 폴더
crop_list = os.listdir(crop_path)
save_path = '/content/drive/MyDrive/23-1 CV프로젝트 1팀/add_webtoon/waifu/' # 저장 폴더 지정
save_list = os.listdir(save_path)

In [6]:
my_list = []

for file in crop_list:
  if file not in save_list:
    my_list.append(file)

In [7]:
my_list

['작전명 순정']

In [8]:
# save path에 폴더 생성(한번만 돌리기~)

for webtoon in my_list:
  os.mkdir(os.path.join(save_path,  webtoon))

# Using the Model (Running Inference)

In [9]:
def run_waifu2x(input_file, models, output_file):
  model = UpConv_7()
  img = Image.open(input_file).convert("RGB")
  img_resized = img.resize((img.size[0] * 2, img.size[1] * 2), Image.BICUBIC)
  img_t = to_tensor(img_resized).unsqueeze(0)
  # put the original image in the final tensor
  # final = img_t
  for model_file in models:
    model.load_pre_train_weights(json_file=model_file)

    # overlapping split
    # if input image is too large, then split it into overlapped patches
    # details can be found at [here](https://github.com/nagadomi/waifu2x/issues/238)
    img_splitter = ImageSplitter(seg_size=64, scale_factor=2, boarder_pad_size=3)
    img_patches = img_splitter.split_img_tensor(img, scale_method=None, img_pad=0)
    with torch.no_grad():
        out = [model(i) for i in img_patches]
    img_upscale = img_splitter.merge_img_tensor(out)

    # # concat the upscaled image to the final tensor
    # final = torch.cat([final, img_upscale])

  # save the image using the torchvision util that allows us to draw a grid of images from a tensor
  save_image(img_upscale, output_file, nrow=(1+len(models)))

In [10]:
for webtoon in my_list:
  first_path = crop_path + webtoon + '/'
  final_path = save_path + webtoon + '/'

  print("---------------" + webtoon + " 시작!---------------")

  file_list = os.listdir(first_path)

  for file in file_list:
    image_path = first_path + file

    run_waifu2x(image_path, ['model_check_points/Upconv_7/noise2_scale2.0x_model.json'], \
                final_path + file)

  print("---------------" + webtoon + " 끝!---------------")

---------------작전명 순정 시작!---------------
---------------작전명 순정 끝!---------------
